# <h1 style="color:#d82036">Bios Slides Parser</h1>

**Example of a Bio:**

![Example of a Bio](bio.png "Example of a Bio")

### Desired Data ouput

```json
{
    "user_id": ""
    "user_name": ""
    "overview": ""
    "core_focus": ""
    "experiences": {"org": "", "role":"", "description": ""}
    "key_skills": ""[]
    "tech_stack": ""[]
    "key_clients": ""
}
```

### Importing libraries

In [5]:
import pandas as pd
import json
from uuid import uuid4

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Import slides' JSON

In future we would like to directly integrate to google slides api

In [6]:
with open('./slides-13.09.json') as t:
    data = json.load(t)

### Slide Function

In [7]:
def slides_parser(raw_presentation):
    slides = pd.DataFrame(raw_presentation)
    return slides

Processing of slides + Deletion of skipped slides

In [11]:
temp_slides = slides_parser(data['slides'])
ids = []
for ind, i in enumerate(temp_slides['slideProperties']):
    try:
        if i['isSkipped'] == True:
            ids.append(ind)
    except:
        continue

slides = temp_slides.drop(ids)
print("No. of objects: " + "\033[1m"+str(slides.size))

No. of objects: 404



### Slide parser function

In [142]:
bio_objects = {}

In [143]:
def text_element_parser(text_elements, page_element_no, slide_no):
    bio_objects[str(slide_no)][str(page_element_no)] = []
    for i in text_elements:
        try:
            bio_objects[str(slide_no)][str(page_element_no)].append(str(i['textRun']['content']))
        except:
            continue

def page_elements_parser(slide_page_elements, slide_no):
    bio_objects[str(slide_no)] = {}
    for idx, i in enumerate(slide_page_elements):
        try:
            text_element_parser(i['shape']['text']['textElements'], idx, slide_no)
        except:
            continue

In [144]:
for ind in slides.index:
    try:
        page_elements_parser(slides['pageElements'][ind], ind)
    except:
        continue
print("\033[1m"+ str(len(bio_objects))+ " slides")

101 slides


#### Removal of non Bio slides

In [145]:
for i in bio_objects.copy().keys():
    if len(bio_objects[i]) < 5 or i == '2':
        del bio_objects[i]
print("\033[1m"+ str(len(bio_objects))+ " slides")

90 slides


### Data cleaning

In [16]:
def bios_cleaner(bios_argument):
    bios = {}
    for slide in bios_argument:
        bios[slide] = {}
        try:
            for element in bios_argument[slide]:
                bios[slide][element] = []
                try:
                    for ind, j in enumerate(bios_argument[slide][element]):
                        if j.strip().replace('|','').replace('-','') == '':
                            continue
                        else:
                            bios[slide][element].append(''.join(j.strip().replace("|","").replace("-","").lower()).split())
                except:
                    if bios[slide][element][0].strip().replace('|','').replace('-','') == '':
                        del bios[slide][element]
                    else:
                        bios[slide][element].append(''.join(bios[slide][element][0].strip().replace("|","").replace("-","").lower()).split())
        except:
            raise Exception('Wrong slide structure!')
    return bios

In [18]:
# bio_objects = bios_cleaner(bio_objects)


In [129]:
bio_objects.keys()

dict_keys(['4', '5', '6', '7', '8', '9', '11', '13', '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29', '30', '32', '33', '34', '35', '36', '37', '38', '40', '41', '42', '43', '44', '45', '46', '47', '48', '50', '51', '52', '54', '55', '56', '57', '58', '59', '60', '61', '62', '64', '65', '66', '67', '68', '69', '70', '72', '74', '75', '76', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '99', '100', '101', '102', '103', '104', '105', '106'])

In [147]:
#slide 4
train_data['role_and_title'] = bio_objects['4']['0']
train_data['overview'] = bio_objects['4']['1']
train_data['key_skills'] = bio_objects['4']['2']
train_data['core_focus'] = bio_objects['4']['3']
train_data['clients'] = bio_objects['4']['4']
train_data['industries'] = bio_objects['4']['5']
train_data['tech_stack'] = bio_objects['4']['6']
train_data['name'] = bio_objects['4']['7']
train_data['experiences'] = [bio_objects['4']['9']]
train_data['experiences'].append(bio_objects['4']['10'])
train_data['experiences'].append(bio_objects['4']['11'])

In [148]:
train_data

{'role_and_title': ['Squad Lead AND Board Game Geek\n'],
 'overview': ['Technical delivery lead and Agile coach with over 10 years of experience in digital and professional services\n'],
 'key_skills': ['Agile Coaching and Delivery Management\n'],
 'core_focus': ['Agile/Scrum, Coaching, Career Development, Delivery Management\n'],
 'clients': ['Avios\n',
  'Costa Coffee\n',
  'Diageo\n',
  'Expedia\n',
  'Ruffer\n',
  'Taylor & Francis\n',
  'Travis Perkins\n'],
 'industries': ['Academic Publishing\n',
  'B2B and B2C\n',
  'Financial Services\n',
  'Food and Drink\n',
  'Hospitality\n',
  'Real Estate\n',
  'Retail\n',
  'Travel\n'],
 'tech_stack': ['Scrum @ Scale Certified Practitioner\n',
  'Professional Scrum Master I Certified\n',
  'Jira\n',
  '\n'],
 'name': ['Alex Minssieux\n'],
 'experiences': [['Agile Coach and Scrum Master\n',
   'Ruffer\n',
   '\n',
   'Alex provided Agile coaching and support across 7 dev teams, which included developing and delivering an 8-week Agile train

### Learning

Training data (manual approach)

In [146]:
train_data = {}

In [50]:
#slide 4
train_data['role_and_title'] = bio_objects['4']['0']
train_data['overview'] = bio_objects['4']['1']
train_data['key_skills'] = bio_objects['4']['2']
train_data['core_focus'] = bio_objects['4']['3']
train_data['clients'] = bio_objects['4']['4']
train_data['industries'] = bio_objects['4']['5']
train_data['tech_stack'] = bio_objects['4']['6']
train_data['name'] = bio_objects['4']['7']
train_data['experiences'] = [bio_objects['4']['9']]
train_data['experiences'].append(bio_objects['4']['10'])
train_data['experiences'].append(bio_objects['4']['11'])

#slide 11
train_data['role_and_title'].append(bio_objects['11']['0'])
train_data['overview'].append(bio_objects['11']['1'])
train_data['core_focus'].append(bio_objects['11']['2'])
train_data['key_skills'].append(bio_objects['11']['3'])
train_data['clients'].append(bio_objects['11']['4'])
train_data['industries'].append(bio_objects['11']['5'])
train_data['tech_stack'].append(bio_objects['11']['6'])
train_data['name'].append(bio_objects['11']['7'])
train_data['experiences'].append([bio_objects['11']['9']])
train_data['experiences'].append(bio_objects['11']['10'])
train_data['experiences'].append(bio_objects['11']['11'])

#slide 16
train_data['role_and_title'].append(bio_objects['16']['0'])
train_data['overview'].append(bio_objects['16']['1'])
train_data['core_focus'].append(bio_objects['16']['2'])
train_data['key_skills'].append(bio_objects['16']['3'])
train_data['clients'].append(bio_objects['16']['4'])
train_data['industries'].append(bio_objects['16']['5'])
train_data['tech_stack'].append(bio_objects['16']['6'])
train_data['name'].append(bio_objects['16']['7'])
train_data['experiences'].append([bio_objects['16']['8']])
train_data['experiences'].append(bio_objects['16']['9'])
train_data['experiences'].append(bio_objects['16']['10'])

#slide 28
train_data['role_and_title'].append(bio_objects['28']['8'])
train_data['overview'].append(bio_objects['28']['4'])
train_data['key_skills'].append(bio_objects['28']['6'])
train_data['core_focus'].append(bio_objects['28']['5'])
train_data['clients'].append(bio_objects['28']['12'])
train_data['industries'].append(bio_objects['28']['11'])
train_data['tech_stack'].append(bio_objects['28']['7'])
train_data['name'].append(bio_objects['28']['3'])
train_data['experiences'].append([bio_objects['28']['0']])
train_data['experiences'].append(bio_objects['28']['1'])
train_data['experiences'].append(bio_objects['28']['2'])

#slide 29
train_data['role_and_title'].append(bio_objects['29']['9'])
train_data['overview'].append(bio_objects['29']['7'])
train_data['key_skills'].append(bio_objects['29']['10'])
train_data['core_focus'].append(bio_objects['29']['11'])
train_data['clients'].append(bio_objects['29']['0'])
train_data['industries'].append(bio_objects['29']['1'])
train_data['tech_stack'].append(bio_objects['29']['2'])
train_data['name'].append(bio_objects['29']['8'])
train_data['experiences'].append([bio_objects['29']['4']])
train_data['experiences'].append(bio_objects['29']['5'])
train_data['experiences'].append(bio_objects['29']['13'])

#slide 33
train_data['role_and_title'].append(bio_objects['33']['10'])
train_data['overview'].append(bio_objects['33']['6'])
train_data['key_skills'].append(bio_objects['33']['8'])
train_data['core_focus'].append(bio_objects['33']['7'])
train_data['clients'].append(bio_objects['33']['5'])
train_data['industries'].append(bio_objects['33']['4'])
train_data['tech_stack'].append(bio_objects['33']['3'])
train_data['name'].append(bio_objects['33']['11'])
train_data['experiences'].append([bio_objects['33']['0']])
train_data['experiences'].append(bio_objects['33']['1'])
train_data['experiences'].append(bio_objects['33']['2'])

#slide 40
train_data['role_and_title'].append(bio_objects['40']['0'])
train_data['overview'].append(bio_objects['40']['1'])
train_data['key_skills'].append(bio_objects['40']['3'])
train_data['core_focus'].append(bio_objects['40']['2'])
train_data['clients'].append(bio_objects['40']['4'])
train_data['industries'].append(bio_objects['40']['5'])
train_data['tech_stack'].append(bio_objects['40']['6'])
train_data['name'].append(bio_objects['40']['7'])
train_data['experiences'].append([bio_objects['40']['9']])
train_data['experiences'].append(bio_objects['40']['10'])
train_data['experiences'].append(bio_objects['40']['11'])

#slide 65
train_data['role_and_title'].append(bio_objects['65']['0'])
train_data['overview'].append(bio_objects['65']['1'])
train_data['key_skills'].append(bio_objects['65']['3'])
train_data['core_focus'].append(bio_objects['65']['2'])
train_data['clients'].append(bio_objects['65']['4'])
train_data['industries'].append(bio_objects['65']['5'])
train_data['tech_stack'].append(bio_objects['65']['6'])
train_data['name'].append(bio_objects['65']['7'])
train_data['experiences'].append([bio_objects['65']['9']])
train_data['experiences'].append(bio_objects['65']['10'])
train_data['experiences'].append(bio_objects['65']['11'])

#slide 66
train_data['role_and_title'].append(bio_objects['66']['10'])
train_data['overview'].append(bio_objects['66']['11'])
train_data['key_skills'].append(bio_objects['66']['13'])
train_data['core_focus'].append(bio_objects['66']['12'])
train_data['clients'].append(bio_objects['66']['6'])
train_data['industries'].append(bio_objects['66']['7'])
train_data['tech_stack'].append(bio_objects['66']['8'])
train_data['name'].append(bio_objects['66']['9'])
train_data['experiences'].append([bio_objects['66']['3']])
train_data['experiences'].append(bio_objects['66']['4'])
train_data['experiences'].append(bio_objects['66']['5'])

#slide 72
train_data['role_and_title'].append(bio_objects['72']['0'])
train_data['overview'].append(bio_objects['72']['1'])
train_data['key_skills'].append(bio_objects['72']['3'])
train_data['core_focus'].append(bio_objects['72']['2'])
train_data['clients'].append(bio_objects['72']['4'])
train_data['industries'].append(bio_objects['72']['5'])
train_data['tech_stack'].append(bio_objects['72']['6'])
train_data['name'].append(bio_objects['72']['7'])
train_data['experiences'].append([bio_objects['72']['9']])
train_data['experiences'].append(bio_objects['72']['10'])
train_data['experiences'].append(bio_objects['72']['11'])
train_data['experiences'].append(bio_objects['72']['12'])

#slide 84
train_data['role_and_title'].append(bio_objects['84']['3'])
train_data['overview'].append(bio_objects['84']['4'])
train_data['key_skills'].append(bio_objects['84']['6'])
train_data['core_focus'].append(bio_objects['84']['5'])
train_data['clients'].append(bio_objects['84']['7'])
train_data['industries'].append(bio_objects['84']['8'])
train_data['tech_stack'].append(bio_objects['84']['9'])
train_data['name'].append(bio_objects['84']['10'])
train_data['experiences'].append([bio_objects['84']['0']])
train_data['experiences'].append(bio_objects['84']['1'])
train_data['experiences'].append(bio_objects['84']['2'])

#slide 90
train_data['role_and_title'].append(bio_objects['90']['10'])
train_data['overview'].append(bio_objects['90']['11'])
train_data['key_skills'].append(bio_objects['90']['13'])
train_data['core_focus'].append(bio_objects['90']['12'])
train_data['clients'].append(bio_objects['90']['6'])
train_data['industries'].append(bio_objects['90']['7'])
train_data['tech_stack'].append(bio_objects['90']['8'])
train_data['name'].append(bio_objects['90']['9'])
train_data['experiences'].append([bio_objects['90']['3']])
train_data['experiences'].append(bio_objects['90']['4'])
train_data['experiences'].append(bio_objects['90']['5'])

#slide 91
train_data['role_and_title'].append(bio_objects['91']['5'])
train_data['overview'].append(bio_objects['91']['1'])
train_data['key_skills'].append(bio_objects['91']['3'])
train_data['core_focus'].append(bio_objects['91']['2'])
train_data['clients'].append(bio_objects['91']['6'])
train_data['industries'].append(bio_objects['91']['7'])
train_data['tech_stack'].append(bio_objects['91']['8'])
train_data['name'].append(bio_objects['91']['0'])
train_data['experiences'].append([bio_objects['91']['9']])
train_data['experiences'].append(bio_objects['91']['10'])
train_data['experiences'].append(bio_objects['91']['11'])

In [51]:
key_clients = ['Squad', 'Whitbread', 'London\n', 'Lloyds of London','Department', 'Azzurri\n', 'Azzurri', 'Azzurri Group', 'Azzurri Group\n', 'Loyds', 'Walters\n', 'Openr\n', 'Openr', 'OpenR', ' Oxford','Oxford','Aviva', 'Group', 'Home Office', 'Citi', 'S&P', 'AXA', 'TUI', 'Mopro', 'FWD', 'FinTech/InsurTech', 'e-Commerce', 'Ineos', 'Invesco', 'TFL', 'Taylor', 'Royal', 'Shawbrook', 'ByBits', 'Berry', 'Mporium', 'Lego', 'Nanopore', 'WorldRemit', 'WorldRemit', 'Lenio', 'Splink', 'HSSMI', 'Ironfly', 'Ford', 'Vida', 'MHHS', 'Jet', 'Cancer', 'Bharat', 'Idea', 'Audi', 'Oxford Nanopore', 'Zenith', 'Zenith', 'British', 'United', 'IAG', 'Wine', 'Dining', 'Solotel', 'Say', 'MOD', 'Bayswater', 'BNP', 'Honda', 'Rakuten', 'Automobile', 'RSA', 'BAE', 'Robert Walters  \n', 'Walters', 'Robert', 'Titan', 'Digital']

key_techs = ['Delivery', 'Product','Product Delivery','Agile/Scrum','AWS', 'Tech', 'Competitor Analysis', 'Analysis', 'Scrum', 'Agile', 'Figma', 'Full', 'Full-Stack', 'Jira\n', 'Jira', 'Front', 'Front-End', 'Front-end', 'Frontend', 'Back', 'Back-end', 'Back-End', 'Backend', 'Engineer', 'Product', 'Engineer', 'End', 'Node.js\n', 'Postgres\n','Infrastructure', 'Experience', 'Strategic', 'UX', 'UI', 'Design', 'Figma', 'Finance', 'User', 'CMS', 'Marketing', 'Mobile', 'e-commerce', 'E-Commerce', 'E-commerce', 'Business', 'Address', 'Language', 'SQL', 'DevOps', 'Customer', 'NodeJS', 'INDUSTRY', 'Loyalty', 'KEY', 'B2B', 'Brand', 'CRM', 'Typescript', 'Kafka', 'Tester', 'eCommerce',  ' | Coaching', 'DevOps/SD/Product', 'Software', 'Web', 'Infrastructure,', 'NA', 'Automated', 'Machine', 'Node.js', 'Postgres', 'Card', 'Systems', 'Hi-Fi', 'design', 'Design', 'Associate', 'Product Analysis & Delivery', 'Digital', 'ideas', 'products', ' Product Developer AND Sudoku Solver']

key_industries = ['Events\n', 'Hospitality\n', 'Hospitality','Music\n', 'B2C\n', 'Retail\n', 'Retail', 'Casual', 'Telecommunications', 'Gambling', 'Gmabling\n', 'Travel', 'Public', 'UK', 'FinTech', 'Media', 'Insurance', 'Defence', 'Banking', 'Health', 'Education', 'Healthcare', 'Real', 'Professional','Hotels', 'Insurtech', 'Games', 'International', 'Recruitment', 'Fintech', 'Financial', 'Pharmaceutical', 'Charity', 'Railway', 'Energy', 'Manufacturing', 'Oil', 'Aviation', 'Leisure', 'Aerospace', 'Military', 'Advertisement', 'Music']

In [52]:
train_data['clients'].append(key_clients)
train_data['tech_stack'].append(key_techs)
train_data['industries'].append(key_industries)

In [53]:
categories = ['role_and_title',
 'overview',
 'key_skills',
 'core_focus',
 'clients',
 'industries',
 'tech_stack',
 'name',
 'experiences']

train_data_copy = {}

for i in categories:
    train_data_copy[i] = []

train_data_copy

{'role_and_title': [],
 'overview': [],
 'key_skills': [],
 'core_focus': [],
 'clients': [],
 'industries': [],
 'tech_stack': [],
 'name': [],
 'experiences': []}

In [54]:
def list_cleaner(key):
    for i in train_data[key]:
        if type(i) == list:
            try:
                for n in i:
                    if type(n) == list:
                        for j in n:
                            if len(j) > 1 and not (j.strip() == ''):
                                train_data_copy[key].append(j.strip().lower())                            
                    elif len(n) > 1 and not (n.strip() == ''):
                        train_data_copy[key].append(n.strip().lower())
            except:
                continue
        else:
            if len(i) > 1 and not (i.strip() == ''):
                train_data_copy[key].append(i.strip().lower())
#     return train_data_copy[key]

In [55]:
for i in categories:
    list_cleaner(i)

In [56]:
train_data = train_data_copy

## Learning algorithms

In [57]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

### Combine text data and labels

In [58]:
del train_data['experiences']
del train_data['name']
del train_data['core_focus']
del train_data['role_and_title']
del train_data['key_skills']
del train_data['overview']

In [59]:
# train_data

In [60]:
X = []
y = []

for category, texts in train_data.items():
    X.extend(texts)
    y.extend([category] * len(texts))

### Split data into training and testing sets

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.1,
                                                    random_state=42)

### Vectorize text using TF-IDF

In [62]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [1]:
X_train_tfidf

NameError: name 'X_train_tfidf' is not defined

### Train a Multinomial Naive Bayes classifier

In [63]:
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

### Make predictions

In [64]:
y_pred = classifier.predict(X_test_tfidf)

### Evaluate the model

In [65]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Accuracy: 0.73
Classification Report:
              precision    recall  f1-score   support

     clients       1.00      0.62      0.77        16
  industries       1.00      0.44      0.62         9
  tech_stack       0.59      1.00      0.74        16

    accuracy                           0.73        41
   macro avg       0.86      0.69      0.71        41
weighted avg       0.84      0.73      0.73        41



### Model evaluation with random data

In [139]:
sample_data = ['Health']
sample_data_tfid = vectorizer.transform(sample_data)

In [140]:
sample_data_prediction = classifier.predict(sample_data_tfid)

In [141]:
for text, predicted_label in zip(sample_data, sample_data_prediction):
    print("Text: " + str(text))
    print("Predicted Label: " +str(predicted_label))
    print()

Text: Health
Predicted Label: industries



### ANDi's Names

In [8]:
bio_objects = {}

In [12]:
def text_element_parser(text_elements, page_element_no, slide_no):
    bio_objects[str(slide_no)][str(page_element_no)] = []
    for i in text_elements:
        try:
            bio_objects[str(slide_no)][str(page_element_no)].append(str(i['textRun']['content']))
        except:
            continue

def page_elements_parser(slide_page_elements, slide_no):
    bio_objects[str(slide_no)] = {}
    for idx, i in enumerate(slide_page_elements):
        try:
            text_element_parser(i['shape']['text']['textElements'], idx, slide_no)
        except:
            continue

In [13]:
for ind in slides.index:
    try:
        page_elements_parser(slides['pageElements'][ind], ind)
    except:
        continue

Remove non Bio slides

In [14]:
for i in bio_objects.copy().keys():
    if len(bio_objects[i]) < 5 or i == '2':
        del bio_objects[i]

Name identifier function

In [23]:
def name_identifier(bio_elements):
    for i in bio_elements:
        if len(bio_elements[i]) <= 4:
            for j in bio_elements[i]:
                if 'AND' not in j and len(j.split(' ')) < 5 and len(j.split('\n')) < 4 and not j.isspace()\
                and ((len(set(key_clients) & set(j.split('\n'))) == 0) and (len(set(key_techs) & set(j.split('\n'))) == 0) and (len(set(key_industries) & set(j.split('\n'))) == 0))\
                and ((len(set(key_clients) & set(j.split(' '))) == 0) and (len(set(key_techs) & set(j.split(' '))) == 0) and (len(set(key_industries) & set(j.split(' '))) == 0)) :
                    return j
        else:
            continue

AND Title Identifier

In [39]:
def and_title_identifier(bio_elements):
    for i in bio_elements:
        if len(bio_elements[i]) <= 4:
            for j in bio_elements[i]:
                if 'AND' in j and 'AND Digital' not in j and len(j.split(' ')) <= 15 and not j.isspace():
                    return j[j.find('AND'):].replace('\n','')
        else:
            continue

User records

In [40]:
users = []
            
for i in bio_objects.keys():
    name = name_identifier(bio_objects[i]).rstrip().split('\n')[0].split(' ')
    full_name = name_identifier(bio_objects[i]).rstrip()
    and_title = and_title_identifier(bio_objects[i])
    temp_id = str(uuid4())
    users.append({'key': 'USER#'+ temp_id,
                  'id': 'USER#' + temp_id,
                  'firstName': name[0],
                  "lastName": name[-1],
                  'name': full_name,
                  'and_tile': and_title
                 })
    
users = pd.DataFrame(users).drop_duplicates(subset=['name'])

In [46]:
users[37:48]

,key,id,firstName,lastName,name,and_tile
40,USER#4dc1e101-08ea-45c7-9896-85e4ac64504b,USER#4dc1e101-08ea-45c7-9896-85e4ac64504b,Adam,Green,Adam Green,AND Friendly Host
41,USER#9a6548f9-d12b-4d8f-bc76-b36655047581,USER#9a6548f9-d12b-4d8f-bc76-b36655047581,George,Velimachitis,George Velimachitis,AND Table Tennis Enthusiast++
42,USER#161b7647-9a33-488f-bc81-22088a013c22,USER#161b7647-9a33-488f-bc81-22088a013c22,Vijay,Mistry,Vijay Mistry,AND Amateur Photographer
43,USER#eae1a428-33d7-4dca-93ca-33031ccdadf7,USER#eae1a428-33d7-4dca-93ca-33031ccdadf7,Catalin,Ghinescu,Catalin Ghinescu,AND Food Nerd
44,USER#f97bf21d-22af-421a-9beb-5859c7c1a4ab,USER#f97bf21d-22af-421a-9beb-5859c7c1a4ab,Ashraf,Nazar,Ashraf Nazar,AND Curious Questioner
45,USER#2c3bfad3-7d7e-4e3f-9a2a-ad2c5cfd0f50,USER#2c3bfad3-7d7e-4e3f-9a2a-ad2c5cfd0f50,Umair,Bashir,Umair Bashir,AND Volunteer
46,USER#4602378b-938e-43f2-a2c5-3fce8278b046,USER#4602378b-938e-43f2-a2c5-3fce8278b046,Kieran,Osgood,Kieran Osgood,AND Manga Reader
47,USER#3f6ac902-fc50-46c3-8170-bfac775fe992,USER#3f6ac902-fc50-46c3-8170-bfac775fe992,May,Ho,May Ho,AND Sneaker Head
48,USER#cc2b0f37-1412-4674-9516-16001863afa6,USER#cc2b0f37-1412-4674-9516-16001863afa6,Katia,Toboleva,Katia Toboleva,AND Binge Learner
49,USER#05ac3280-a263-4e40-88a1-4bcacf2cf521,USER#05ac3280-a263-4e40-88a1-4bcacf2cf521,Jorge,Baltazar,Jorge Baltazar,AND Joga Bonito Fan


In [42]:
users[50:65]

,key,id,firstName,lastName,name,and_tile
54,USER#ea8aeb94-9107-47bd-b05d-4b21a2d7d7e6,USER#ea8aeb94-9107-47bd-b05d-4b21a2d7d7e6,Issy,Cameron,Issy Cameron,AND Wine Enthusiast
55,USER#9bbb4527-1a69-419b-bb70-b899ca8c5dd3,USER#9bbb4527-1a69-419b-bb70-b899ca8c5dd3,Harry,Wright,Harry Wright,AND Dancing Dad
56,USER#5acbe066-e19c-4524-8187-6764d6c563dd,USER#5acbe066-e19c-4524-8187-6764d6c563dd,Jon,Dwyer,Jon Dwyer,AND Maker
57,USER#beb7a024-8996-42c1-9217-43c34e8797e5,USER#beb7a024-8996-42c1-9217-43c34e8797e5,Patrick,Brown,Patrick Brown,AND Wild Swimmer
58,USER#9a31206f-67f7-4bc7-a5c4-3495a497bd63,USER#9a31206f-67f7-4bc7-a5c4-3495a497bd63,Sonia,Choudhury,Sonia Choudhury,AND Adventure Seeker
59,USER#24b7b068-fc30-47b7-a3e8-031107246f24,USER#24b7b068-fc30-47b7-a3e8-031107246f24,Permjit,Ghatorre,Permjit Singh Ghatorre,AND Accidental Hero
60,USER#bdbe7719-93af-4780-a10f-eec65d7788ba,USER#bdbe7719-93af-4780-a10f-eec65d7788ba,Mamta,Sharma,Mamta Sharma,AND Travel Enthusiast
61,USER#48131a75-52df-4125-b54c-dd728e7bb29d,USER#48131a75-52df-4125-b54c-dd728e7bb29d,Andrew,Ajube,Andrew Ajube,AND 400m Hurdler
62,USER#88c40614-e126-462a-ad07-2159e694d26e,USER#88c40614-e126-462a-ad07-2159e694d26e,Minhye,Lee,Minhye Lee,AND Matchmaker
63,USER#814afd31-3aa4-4b2c-b776-4fa5f2e4ecaa,USER#814afd31-3aa4-4b2c-b776-4fa5f2e4ecaa,Bruce,Horne,Bruce Horne,AND Hornebre


Attaching users to BU

In [47]:
users_bu = []
for i in users['key']:
        users_bu.append({'key': i, 'id': 'BUSINESS_UNIT#CLUBTURING', 'name': 'Club Turing'})


users_bu = pd.DataFrame(users_bu)
users = pd.concat([users, users_bu])

In [48]:
users[users['key'] == users[users['firstName'] == 'Ashraf']['key'].values[0]]

,key,id,firstName,lastName,name,and_tile
44,USER#f97bf21d-22af-421a-9beb-5859c7c1a4ab,USER#f97bf21d-22af-421a-9beb-5859c7c1a4ab,Ashraf,Nazar,Ashraf Nazar,AND Curious Questioner
41,USER#f97bf21d-22af-421a-9beb-5859c7c1a4ab,BUSINESS_UNIT#CLUBTURING,NaN,NaN,Club Turing,NaN


### key_skills

### core_focus

### tech_stack

### industries

### key_clients

### Experiences

Org.

Role

Description

Loop through each slide making them a single record with only the text they got

Apply modeling to each record to identify what is what

In [ ]:
with open('./caio_garcia.json', 'r') as f:
  data = json.load(f)

In [ ]:
df = pd.DataFrame(data['pageElements'])

In [ ]:
tmp = []
for i in range(len(df['shape'])):
    tmp.append(df['shape'][i])
# len(tmp)

In [ ]:
for i in range(len(tmp)):
    try:
        for j in range(len(tmp[i]['text']['textElements'])):
            try:
                if j ==1:
                    print("%d: " % (j) + tmp[i]['text']['textElements'][j]['textRun']['content'])
                else:
                    print(tmp[i]['text']['textElements'][j]['textRun']['content'])
            except:
                continue
    except:
        continue

In [ ]:
# def runner_function(arg):
#     arr = []
#     for i in range(len(df[arg])):
#         arr.append(df[arg][i])
    
#     for j in range(len(arr)):
#         try:
            

        
        
        
    
#     return arr
            
        
        
        
        
        
        
        
        
        
        
        
        
        
        
# #         try:
# #             for j in range(len(df[arg][i])):
# #                 try:
# #                     arr.append(df[arg][i]['text']['textElements'][j]['textRun']['content'])
# #                 except:
# #                     continue
# #         except:
# #             continue
# #     return arr

In [ ]:
# runner_function('shape')

In [ ]:
with open('./musab_adam_slide.json', 'r') as f:
  data = json.load(f)

In [ ]:
df = pd.DataFrame(data['pageElements'])

In [ ]:
tmp = []
for i in range(len(df['shape'])):
    tmp.append(df['shape'][i])
len(tmp)

In [ ]:
for i in range(len(tmp)):
    try:
        for j in range(len(tmp[i]['text']['textElements'])):
            try:
                if j ==1:
                    print("%d: " % (j) + tmp[i]['text']['textElements'][j]['textRun']['content'])
                else:
                    print(tmp[i]['text']['textElements'][j]['textRun']['content'])
            except:
                continue
    except:
        continue